In [1]:
# 라이브러리 호출
import requests
import json
from bs4 import BeautifulSoup as bts
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import os

In [23]:
# 네이버뉴스 링크를 수집하고 데이터프레임으로 반환하는 함수 생성
def NaverNewsLink(searchWord, startNo = 1):
    
    # HTTP 요청 실행
    # 쿼리 문자열: 정렬(sort)-관련도순, 기간(pd)-전체
    res = requests.get(
        url = 'https://s.search.naver.com/p/newssearch/search.naver',         
        params = {
            'nqx_theme': {'theme':{'main':{'name':'sports_event'},'sub':[{'name':'issue'}]}},
            'query': searchWord,
            'sort': 2,
            'spq': 3,
            'pd': 5,
            'start': startNo,
            'where': 'news_tab_api',
        }
    )
    
    # JSON 형태의 문자열을 딕셔너리로 변환
    dic = json.loads(s = res.text)
    
    # 딕셔너리의 각 원소를 bs4.BeautifulSoup 객체로 변환
    # [주의] HTTP 응답 바디 문자열에 공백이 있으면 items의 원소로 추가되므로 삭제해야 함
    items = [bts(markup = i.strip(), features = 'html.parser') for i in dic['contents']]
    
    # 뉴스 링크를 포함하는 HTML 요소를 원소로 갖는 links 생성
    links = [item.select('a.info:last-child') for item in items]

    # 기사 작성 일시를 포함하는 HTML 요소를 원소로 갖는 dates 생성
    dates = [item.select('span.info')[0].text for item in items if len(item.select('span.info')) > 0]
    
    # 언론사, 제목 및 네이버뉴스 링크를 원소로 갖는 데이터프레임 생성
    newsList = pd.DataFrame(data = {
        'press': [item.select(selector = 'a.press')[0].text for item in items], 
        'title': [item.select(selector = 'a.news_tit')[0].text for item in items], 
        'nlink': [link[0]['href'] if len(link) >= 1 else np.nan for link in links],
        'date': dates
    })
    
    # 데이터프레임 반환
    return newsList

In [24]:
# 검색어 설정
searchWord = 'SK하이닉스'

In [25]:
# 함수 테스트
NaverNewsLink(searchWord = searchWord)

,press,title,nlink,date
0,핀포인트뉴스,[코스피] 외국인 순매수 TOP5 삼성전자·SK하이닉스·현대차·삼성전기·...,NaN,2023.10.18.
1,글로벌이코노믹,"이란 최고지도자 하메네이 ""이스라엘 경고""",NaN,2023.10.18.
2,포인트데일리,"시스템반도체 관련주, '맑음' 하나마이크론·테크윙·한미반도체... '흐...",NaN,2023.10.18.
3,뉴스1,"美, 대중국 반도체 수출통제 강화…저사양 AI칩도 포함(종합)",https://n.news.naver.com/mnews/article/421/000...,2023.10.18.
4,서울경제언론사 선정,"SK하이닉스, 기옥시아·WD 합병 반대…소프트뱅크에 협력 타진",https://n.news.naver.com/mnews/article/011/000...,2023.10.18.
5,핀포인트뉴스,현대차·삼성전기 등 외국인 순매수 활발…기술력 지수 '저평가',NaN,2023.10.18.
6,동아일보언론사 선정,"美, 저사양 AI 반도체도 중국 수출 차단",https://n.news.naver.com/mnews/article/020/000...,2023.10.18.
7,핀포인트뉴스,비즈니스온·SK하이닉스 등 52주 최고가 근접…기술력 지수는?,NaN,2023.10.18.
8,뉴시스언론사 선정,반도체株 날개 달았다…7만전자 13만닉스 '훨훨',https://n.news.naver.com/mnews/article/003/001...,2023.10.18.
9,컨슈머타임스,"박덕수 인천시 행정부시장, 충칭시 부시장과 '교류 의지' 다져",NaN,2023.10.18.


In [26]:
# 수집 건수 설정
count = 2000

In [27]:
# 최종 결과를 저장할 빈 데이터프레임 생성
newsList = pd.DataFrame()

# 반복문으로 관련 네이버뉴스 링크 수집
for i in tqdm(range(1, count + 1, 10)):
    
    # 네이버뉴스 링크를 수집하고 df에 할당
    df = NaverNewsLink(searchWord = searchWord, startNo = i)
    
    # 최종 결과에 df에 추가(행이름 삭제)
    newsList = pd.concat(objs = [newsList, df], ignore_index = True)
    
    # 1초간 멈춤
    time.sleep(1)

  0%|          | 0/200 [00:00<?, ?it/s]

In [28]:
# nlink가 결측인 행 삭제
newsList = newsList.loc[newsList['nlink'].notna(), :]

In [29]:
# press에서 '언론사 선정' 삭제
newsList['press'] = newsList['press'].str.replace(pat = '언론사 선정', repl = '')

In [30]:
# newsList의 행 개수 확인
newsList.shape[0]

780

In [31]:
# nlink의 중복 여부(True/False)를 원소로 갖는 dups 생성
# [참고] keep 매개변수에 'first'를 지정하면 첫 번째 중복 건은 False로 반환
dups = newsList['nlink'].duplicated(keep = 'first')

In [32]:
# nlink가 중복인 행이 있으면 해당 행을 삭제
newsList = newsList.loc[~dups, :]

In [33]:
# newsList의 행 개수 확인
newsList.shape[0]

764

In [34]:
# nlink에서 쿼리 문자열 삭제
newsList['nlink'] = newsList['nlink'].str.replace(pat = r'(\?.+)', repl = '', regex = True)

In [35]:
# newsList의 행이름 초기화
newsList = newsList.reset_index(drop = True)

In [36]:
# newsList의 처음 5행 확인
newsList.tail()

,press,title,nlink,date
759,연합뉴스,"HBM 날개 단 SK하이닉스, 3분기 적자 줄였다…D램은 흑자전환(종합)",https://n.news.naver.com/mnews/article/001/001...,2023.10.26.
760,노컷뉴스,"SK하이닉스, 3Q 영업손실 1.8조…적자 '1조' 줄였다",https://n.news.naver.com/mnews/article/079/000...,2023.10.26.
761,파이낸셜뉴스,[상보] SK하이닉스 D램 흑자전환... 적자폭 1兆 감소,https://n.news.naver.com/mnews/article/014/000...,2023.10.26.
762,서울신문,[속보] SK하이닉스 3분기 영업손실 1조 8000억원…D램은 흑자 전환,https://n.news.naver.com/mnews/article/081/000...,2023.10.26.
763,한국경제,SK하이닉스 D램 2분기 만에 흑자전환…3분기 영업손실 1.8조원,https://n.news.naver.com/mnews/article/015/000...,2023.10.26.


In [37]:
# 현재 작업 경로 확인
os.getcwd()

'C:\\Users\\2-13\\Desktop\\DMF\\final\\naver'

In [38]:
# newsList를 pkl 파일로 저장
pd.to_pickle(obj = newsList, filepath_or_buffer = 'Naver_News_List_2.pkl')

In [39]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['.ipynb_checkpoints',
 'crawling.py',
 'data',
 'Naver_News_List.pkl',
 'Naver_News_List_2.pkl',
 'naver_news_samsung.csv',
 'SK하이닉스크롤링.ipynb',
 'Untitled1.ipynb',
 '삼전 크롤링 step2.ipynb',
 '삼전크롤링.ipynb']

## end of document